In [4]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from PIL import Image, ImageTk


min_contour_width = 40
min_contour_height = 40
offset = 10
line_height = 550
video_path = ""




def calculate_evaluation_measures(true_positive, false_positive, false_negative):
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = true_positive / (true_positive + false_positive + false_negative) if (true_positive + false_positive + false_negative) > 0 else 0
    return precision, recall, f1_score, accuracy


class VideoProcessor:
    def __init__(self, root):
        self.root = root
        self.root.title("Vehicle Detection using OpenCV")

        
        self.video_frame = tk.Frame(root)
        self.video_frame.pack(side=tk.LEFT, padx=10, pady=10)

        self.video_label = tk.Label(self.video_frame)
        self.video_label.pack()

        self.btn_upload_video = tk.Button(self.video_frame, text="Upload Video", command=self.upload_video)
        self.btn_upload_video.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_contour = tk.Button(self.video_frame, text="Contour video", command=self.process_contour)
        self.btn_contour.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_haarcascade = tk.Button(self.video_frame, text="Haarcascade video", command=self.process_haarcascade)
        self.btn_haarcascade.pack(side=tk.TOP, padx=5, pady=5)

        self.cap = None

       
        self.precision_label = tk.Label(self.video_frame, text="Precision: ")
        self.precision_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.recall_label = tk.Label(self.video_frame, text="Recall: ")
        self.recall_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.f1_score_label = tk.Label(self.video_frame, text="F1 Score: ")
        self.f1_score_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.true_positive_label = tk.Label(self.video_frame, text="True Positive: ")
        self.true_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_positive_label = tk.Label(self.video_frame, text="False Positive: ")
        self.false_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_negative_label = tk.Label(self.video_frame, text="False Negative: ")
        self.false_negative_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.accuracy_label = tk.Label(self.video_frame, text="Accuracy: ")
        self.accuracy_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")

  
    def upload_video(self):
        global video_path
        file_path = filedialog.askopenfilename()
        if file_path:
            video_path = file_path
            self.cap = cv2.VideoCapture(video_path)
            self.show_video()

   
    def show_video(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (400, 300))
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
            self.video_label.after(10, self.show_video)

    def process_contour(self):
        if self.cap is None:
            print("No video loaded")
            return

        def perform_vehicle_detection(label):
            global matches, cars, video_path
            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print("Error: Unable to open video file.")
                return

            ret, frame1 = cap.read()
            ret, frame2 = cap.read()

            video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            true_positive = 0
            false_positive = 0
            false_negative = 0

            while ret:
                d = cv2.absdiff(frame1, frame2)
                grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)

                blur = cv2.GaussianBlur(grey, (5, 5), 0)

                ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
                dilated = cv2.dilate(th, np.ones((3, 3)))
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

                closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
                contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                for c in contours:
                    x, y, w, h = cv2.boundingRect(c)
                    contour_valid = (w >= min_contour_width) and (h >= min_contour_height)

                    if not contour_valid:
                        continue

                    cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 2)
                    cv2.line(frame1, (0, line_height), (video_width, line_height), (0, 255, 0), 2)
                   
                   

                frame_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame_rgb)
                imgtk = ImageTk.PhotoImage(image=img)

                label.imgtk = imgtk
                label.config(image=imgtk)
                label.update()

                frame1 = frame2
                ret, frame2 = cap.read()

            if not ret:
                false_negative = len([(100, 100, 50, 50)]) - true_positive
                self.update_evaluation_measures(true_positive, false_positive, false_negative)

            cv2.destroyAllWindows()
            cap.release()

        perform_vehicle_detection(self.video_label)

   
    def process_haarcascade(self):
        if self.cap is None:
            print("No video loaded")
            return

        def detect_vehicles_haar(label):
            global video_path
            car_cascade = cv2.CascadeClassifier('cars.xml')

            cap = cv2.VideoCapture(video_path)

            true_positive = 0
            false_positive = 0
            false_negative = 0

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(grey, (5, 5), 0)
                dilated = cv2.dilate(blur, np.ones((3, 3)))

                cars = car_cascade.detectMultiScale(dilated, 1.1, 1)

                for (x, y, w, h) in cars:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    for gt_x, gt_y, gt_w, gt_h in [(100, 100, 50, 50)]:
                        if (x < gt_x + gt_w and x + w > gt_x and
                                y < gt_y + gt_h and y + h > gt_y):
                            true_positive += 1
                        else:
                            false_positive += 1

                false_negative = len([(100, 100, 50, 50)]) - true_positive

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame_rgb)
                imgtk = ImageTk.PhotoImage(image=img)

                label.imgtk = imgtk
                label.config(image=imgtk)
                label.update()

            if not ret:
                self.update_evaluation_measures(true_positive, false_positive, false_negative)

            cap.release()
            cv2.destroyAllWindows()

        detect_vehicles_haar(self.video_label)

    def update_evaluation_measures(self, true_positive, false_positive, false_negative):
        precision, recall, f1_score, accuracy = calculate_evaluation_measures(true_positive, false_positive, false_negative)
       
        self.precision_label.config(text="Precision: {:.2f}".format(precision))
        self.recall_label.config(text="Recall: {:.2f}".format(recall))
        self.f1_score_label.config(text="F1 Score: {:.2f}".format(f1_score))
        self.true_positive_label.config(text="True Positive: {}".format(true_positive))
        self.false_positive_label.config(text="False Positive: {}".format(false_positive))
        self.false_negative_label.config(text="False Negative: {}".format(false_negative))
        self.accuracy_label.config(text="Accuracy: {:.2f}".format(accuracy))


class ImageProcessor:
    def __init__(self, root):
        self.root = root

        self.image_frame = tk.Frame(root)
        self.image_frame.pack(side=tk.RIGHT, padx=10, pady=10)

        self.image_label = tk.Label(self.image_frame)
        self.image_label.pack()

        self.btn_upload_image = tk.Button(self.image_frame, text="Upload Image", command=self.upload_image)
        self.btn_upload_image.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_contour = tk.Button(self.image_frame, text="Contour image", command=self.process_contour)
        self.btn_contour.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_haarcascade = tk.Button(self.image_frame, text="Haarcascade image", command=self.process_haarcascade)
        self.btn_haarcascade.pack(side=tk.TOP, padx=5, pady=5)

        self.image = None

        
        self.precision_label = tk.Label(self.image_frame, text="Precision: ")
        self.precision_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.recall_label = tk.Label(self.image_frame, text="Recall: ")
        self.recall_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.f1_score_label = tk.Label(self.image_frame, text="F1 Score: ")
        self.f1_score_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.true_positive_label = tk.Label(self.image_frame, text="True Positive: ")
        self.true_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_positive_label = tk.Label(self.image_frame, text="False Positive: ")
        self.false_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_negative_label = tk.Label(self.image_frame, text="False Negative: ")
        self.false_negative_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.accuracy_label = tk.Label(self.image_frame, text="Accuracy: ")
        self.accuracy_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")

 
    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = cv2.imread(file_path)
            self.show_image()

    
    def show_image(self):
        img = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.image_label.imgtk = imgtk
        self.image_label.configure(image=imgtk)

   
    def process_contour(self):
        if self.image is None:
            print("No image loaded")
            return

        grey = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(grey, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        contour_output = self.image.copy()

        min_area = 53
        min_aspect_ratio = 0.62
        max_aspect_ratio = 0.95
        valid_contours = []
        for contour in contours:
            area = cv2.contourArea(contour)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if area > min_area and min_aspect_ratio < aspect_ratio < max_aspect_ratio:
                valid_contours.append(contour)
                cv2.rectangle(contour_output, (x, y), (x+w, y+h), (0, 255, 0), 2)

        img = cv2.cvtColor(contour_output, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.image_label.imgtk = imgtk
        self.image_label.config(image=imgtk)

        true_positive = len(valid_contours)  
        false_positive = 0  
        false_negative = len([(100, 100, 50, 50)]) - true_positive  

        precision, recall, f1_score, accuracy = calculate_evaluation_measures(true_positive, false_positive, false_negative)
        self.update_evaluation_measures(precision, recall, f1_score, true_positive, false_positive, false_negative, accuracy)

    def process_haarcascade(self):
        if self.image is None:
            print("No image loaded")
            return

        car_cascade = cv2.CascadeClassifier('cars.xml')
        grey = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (5, 5), 0)
        dilated = cv2.dilate(blur, np.ones((3, 3)))

        cars = car_cascade.detectMultiScale(dilated, 1.1, 1)

        haar_output = self.image.copy()
        for (x, y, w, h) in cars:
            cv2.rectangle(haar_output, (x, y), (x + w, y + h), (0, 0, 255), 2)

        img = cv2.cvtColor(haar_output, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.image_label.imgtk = imgtk
        self.image_label.config(image=imgtk)

        true_positive = len(cars) 
        false_positive = 0  
        false_negative = len([(100, 100, 50, 50)]) - true_positive  
        precision, recall, f1_score, accuracy = calculate_evaluation_measures(true_positive, false_positive, false_negative)
        self.update_evaluation_measures(precision, recall, f1_score, true_positive, false_positive, false_negative, accuracy)

    def update_evaluation_measures(self, precision, recall, f1_score, true_positive, false_positive, false_negative, accuracy):
       
        self.precision_label.config(text="Precision: {:.2f}".format(precision))
        self.recall_label.config(text="Recall: {:.2f}".format(recall))
        self.f1_score_label.config(text="F1 Score: {:.2f}".format(f1_score))
        self.true_positive_label.config(text="True Positive: {}".format(true_positive))
        self.false_positive_label.config(text="False Positive: {}".format(false_positive))
        self.false_negative_label.config(text="False Negative: {}".format(false_negative))
        self.accuracy_label.config(text="Accuracy: {:.2f}".format(accuracy))



def main():
    root = tk.Tk()
    root.title("Vehicle Detection using OpenCV")

    video_processor = VideoProcessor(root)
    image_processor = ImageProcessor(root)

    root.mainloop()

if __name__ == "__main__":
    main()


In [5]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from PIL import Image, ImageTk


min_contour_width = 40
min_contour_height = 40
offset = 10
line_height = 550
matches = []
cars = 0
video_path = ""


def get_centroid(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy


def calculate_evaluation_measures(true_positive, false_positive, false_negative):
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = true_positive / (true_positive + false_positive + false_negative) if (true_positive + false_positive + false_negative) > 0 else 0
    return precision, recall, f1_score, accuracy


class VideoProcessor:
    def __init__(self, root):
        self.root = root
        self.root.title("Vehicle Detection using OpenCV")

        
        self.video_frame = tk.Frame(root)
        self.video_frame.pack(side=tk.LEFT, padx=10, pady=10)

        self.video_label = tk.Label(self.video_frame)
        self.video_label.pack()

        self.btn_upload_video = tk.Button(self.video_frame, text="Upload Video", command=self.upload_video)
        self.btn_upload_video.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_contour = tk.Button(self.video_frame, text="Contour video", command=self.process_contour)
        self.btn_contour.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_haarcascade = tk.Button(self.video_frame, text="Haarcascade video", command=self.process_haarcascade)
        self.btn_haarcascade.pack(side=tk.TOP, padx=5, pady=5)

        self.cap = None

       
        self.precision_label = tk.Label(self.video_frame, text="Precision: ")
        self.precision_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.recall_label = tk.Label(self.video_frame, text="Recall: ")
        self.recall_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.f1_score_label = tk.Label(self.video_frame, text="F1 Score: ")
        self.f1_score_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.true_positive_label = tk.Label(self.video_frame, text="True Positive: ")
        self.true_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_positive_label = tk.Label(self.video_frame, text="False Positive: ")
        self.false_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_negative_label = tk.Label(self.video_frame, text="False Negative: ")
        self.false_negative_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.accuracy_label = tk.Label(self.video_frame, text="Accuracy: ")
        self.accuracy_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")

  
    def upload_video(self):
        global video_path
        file_path = filedialog.askopenfilename()
        if file_path:
            video_path = file_path
            self.cap = cv2.VideoCapture(video_path)
            self.show_video()

   
    def show_video(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (400, 300))
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
            self.video_label.after(10, self.show_video)

    def process_contour(self):
        if self.cap is None:
            print("No video loaded")
            return

        def perform_vehicle_detection(label):
            global matches, cars, video_path
            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print("Error: Unable to open video file.")
                return

            ret, frame1 = cap.read()
            ret, frame2 = cap.read()

            video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            true_positive = 0
            false_positive = 0
            false_negative = 0

            while ret:
                d = cv2.absdiff(frame1, frame2)
                grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)

                blur = cv2.GaussianBlur(grey, (5, 5), 0)

                ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
                dilated = cv2.dilate(th, np.ones((3, 3)))
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

                closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
                contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                for c in contours:
                    x, y, w, h = cv2.boundingRect(c)
                    contour_valid = (w >= min_contour_width) and (h >= min_contour_height)

                    if not contour_valid:
                        continue

                    cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 2)
                    cv2.line(frame1, (0, line_height), (video_width, line_height), (0, 255, 0), 2)
                    centroid = get_centroid(x, y, w, h)
                    matches.append(centroid)
                    cv2.circle(frame1, centroid, 5, (0, 255, 0), -1)

                    for point in matches:
                        if point[1] < (line_height + offset) and point[1] > (line_height - offset):
                            true_positive += 1
                            matches.remove(point)

                frame_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame_rgb)
                imgtk = ImageTk.PhotoImage(image=img)

                label.imgtk = imgtk
                label.config(image=imgtk)
                label.update()

                frame1 = frame2
                ret, frame2 = cap.read()

            if not ret:
                false_negative = len([(100, 100, 50, 50)]) - true_positive
                self.update_evaluation_measures(true_positive, false_positive, false_negative)

            cv2.destroyAllWindows()
            cap.release()

        perform_vehicle_detection(self.video_label)

   
    def process_haarcascade(self):
        if self.cap is None:
            print("No video loaded")
            return

        def detect_vehicles_haar(label):
            global video_path
            car_cascade = cv2.CascadeClassifier('cars.xml')

            cap = cv2.VideoCapture(video_path)

            true_positive = 0
            false_positive = 0
            false_negative = 0

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(grey, (5, 5), 0)
                dilated = cv2.dilate(blur, np.ones((3, 3)))

                cars = car_cascade.detectMultiScale(dilated, 1.1, 1)

                for (x, y, w, h) in cars:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    for gt_x, gt_y, gt_w, gt_h in [(100, 100, 50, 50)]:
                        if (x < gt_x + gt_w and x + w > gt_x and
                                y < gt_y + gt_h and y + h > gt_y):
                            true_positive += 1
                        else:
                            false_positive += 1

                false_negative = len([(100, 100, 50, 50)]) - true_positive

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame_rgb)
                imgtk = ImageTk.PhotoImage(image=img)

                label.imgtk = imgtk
                label.config(image=imgtk)
                label.update()

            if not ret:
                self.update_evaluation_measures(true_positive, false_positive, false_negative)

            cap.release()
            cv2.destroyAllWindows()

        detect_vehicles_haar(self.video_label)

    def update_evaluation_measures(self, true_positive, false_positive, false_negative):
        precision, recall, f1_score, accuracy = calculate_evaluation_measures(true_positive, false_positive, false_negative)
       
        self.precision_label.config(text="Precision: {:.2f}".format(precision))
        self.recall_label.config(text="Recall: {:.2f}".format(recall))
        self.f1_score_label.config(text="F1 Score: {:.2f}".format(f1_score))
        self.true_positive_label.config(text="True Positive: {}".format(true_positive))
        self.false_positive_label.config(text="False Positive: {}".format(false_positive))
        self.false_negative_label.config(text="False Negative: {}".format(false_negative))
        self.accuracy_label.config(text="Accuracy: {:.2f}".format(accuracy))


class ImageProcessor:
    def __init__(self, root):
        self.root = root

        self.image_frame = tk.Frame(root)
        self.image_frame.pack(side=tk.RIGHT, padx=10, pady=10)

        self.image_label = tk.Label(self.image_frame)
        self.image_label.pack()

        self.btn_upload_image = tk.Button(self.image_frame, text="Upload Image", command=self.upload_image)
        self.btn_upload_image.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_contour = tk.Button(self.image_frame, text="Contour image", command=self.process_contour)
        self.btn_contour.pack(side=tk.TOP, padx=5, pady=5)

        self.btn_haarcascade = tk.Button(self.image_frame, text="Haarcascade image", command=self.process_haarcascade)
        self.btn_haarcascade.pack(side=tk.TOP, padx=5, pady=5)

        self.image = None

        
        self.precision_label = tk.Label(self.image_frame, text="Precision: ")
        self.precision_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.recall_label = tk.Label(self.image_frame, text="Recall: ")
        self.recall_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.f1_score_label = tk.Label(self.image_frame, text="F1 Score: ")
        self.f1_score_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.true_positive_label = tk.Label(self.image_frame, text="True Positive: ")
        self.true_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_positive_label = tk.Label(self.image_frame, text="False Positive: ")
        self.false_positive_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.false_negative_label = tk.Label(self.image_frame, text="False Negative: ")
        self.false_negative_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")
        self.accuracy_label = tk.Label(self.image_frame, text="Accuracy: ")
        self.accuracy_label.pack(side=tk.TOP, padx=5, pady=5, anchor="center")

 
    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = cv2.imread(file_path)
            self.show_image()

    
    def show_image(self):
        img = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.image_label.imgtk = imgtk
        self.image_label.configure(image=imgtk)

   
    def process_contour(self):
        if self.image is None:
            print("No image loaded")
            return

        grey = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(grey, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        contour_output = self.image.copy()

        min_area = 53
        min_aspect_ratio = 0.62
        max_aspect_ratio = 0.95
        valid_contours = []
        for contour in contours:
            area = cv2.contourArea(contour)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if area > min_area and min_aspect_ratio < aspect_ratio < max_aspect_ratio:
                valid_contours.append(contour)
                cv2.rectangle(contour_output, (x, y), (x+w, y+h), (0, 255, 0), 2)

        img = cv2.cvtColor(contour_output, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.image_label.imgtk = imgtk
        self.image_label.config(image=imgtk)

        true_positive = len(valid_contours)  
        false_positive = 0  
        false_negative = len([(100, 100, 50, 50)]) - true_positive  

        precision, recall, f1_score, accuracy = calculate_evaluation_measures(true_positive, false_positive, false_negative)
        self.update_evaluation_measures(precision, recall, f1_score, true_positive, false_positive, false_negative, accuracy)

    def process_haarcascade(self):
        if self.image is None:
            print("No image loaded")
            return

        car_cascade = cv2.CascadeClassifier('cars.xml')
        grey = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (5, 5), 0)
        dilated = cv2.dilate(blur, np.ones((3, 3)))

        cars = car_cascade.detectMultiScale(dilated, 1.1, 1)

        haar_output = self.image.copy()
        for (x, y, w, h) in cars:
            cv2.rectangle(haar_output, (x, y), (x + w, y + h), (0, 0, 255), 2)

        img = cv2.cvtColor(haar_output, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.image_label.imgtk = imgtk
        self.image_label.config(image=imgtk)

        true_positive = len(cars) 
        false_positive = 0  
        false_negative = len([(100, 100, 50, 50)]) - true_positive  
        precision, recall, f1_score, accuracy = calculate_evaluation_measures(true_positive, false_positive, false_negative)
        self.update_evaluation_measures(precision, recall, f1_score, true_positive, false_positive, false_negative, accuracy)

    def update_evaluation_measures(self, precision, recall, f1_score, true_positive, false_positive, false_negative, accuracy):
       
        self.precision_label.config(text="Precision: {:.2f}".format(precision))
        self.recall_label.config(text="Recall: {:.2f}".format(recall))
        self.f1_score_label.config(text="F1 Score: {:.2f}".format(f1_score))
        self.true_positive_label.config(text="True Positive: {}".format(true_positive))
        self.false_positive_label.config(text="False Positive: {}".format(false_positive))
        self.false_negative_label.config(text="False Negative: {}".format(false_negative))
        self.accuracy_label.config(text="Accuracy: {:.2f}".format(accuracy))



def main():
    root = tk.Tk()
    root.title("Vehicle Detection using OpenCV")

    video_processor = VideoProcessor(root)
    image_processor = ImageProcessor(root)

    root.mainloop()

if __name__ == "__main__":
    main()
